In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm

### Tree and dataframe setup

In [2]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=666,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="te51e00e3dd5e4ee793aed55449806c6d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [3]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [4]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 3.0
𝛽_std = 0.2
𝜓_mean = 0.0
𝜓_std = 0.33

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [136]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,3.081442,0.099620,0.901977
1,0,3.021518,0.272255,1.023924
2,0,3.078410,0.203921,1.056509
3,0,2.861032,0.528396,0.866999
4,0,2.834017,0.087811,1.065575


### Generate crossing data

In [137]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [(get_dist(TREE, i, j) / 2) for (i, j) in zip(a, b)],
})

DATA['b'] = np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)
DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)
DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])
# DATA['error'] = np.random.normal(0.0, 𝜎_std, DATA.shape[0])

# get logits
DATA['logit_b'] = (
    1 / (1 + np.exp(-(DATA.intercept * DATA.b ** DATA.dist)))
)
DATA['logit'] = (
    1 / (1 + np.exp(-(DATA.intercept * DATA.velo ** DATA.dist)))
)
DATA['logit_x'] = (
    1 / (1 + np.exp(-(DATA.intercept * DATA.velo_x ** DATA.dist)))
)

# get RI estimates
DATA['RI_pooled'] = np.random.binomial(n=1, p=DATA.logit_b / DATA.logit_b.max())
DATA['RI_unpooled'] = np.random.binomial(n=1, p=DATA.logit / DATA.logit.max())
DATA['RI_partpooled'] = np.random.binomial(n=1, p=DATA.logit_x / DATA.logit_x.max())

DATA.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.073376,3.052724,3.424599,4.978626,0.043440,0.511784,0.511884,0.512215,0,1,1
1,0,2,0.089748,2.731377,3.034918,4.689863,0.027658,0.507566,0.507638,0.507942,1,0,1
2,0,3,0.129770,2.937952,3.565967,4.706928,0.041294,0.511871,0.512173,0.512619,1,1,1
3,0,4,0.144542,3.221906,3.409337,5.189458,0.061431,0.518179,0.518328,0.519475,1,1,1
4,0,5,0.144542,2.908769,2.847834,4.664815,0.082490,0.524045,0.523972,0.525742,1,1,1


In [138]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,15,73,1.000000,3.152033,3.317109,3.044684,0.047124,0.537066,0.539000,0.535808,1,0,1
1,3,30,0.438896,3.095047,3.595365,4.894121,0.067434,0.527652,0.529527,0.533795,1,1,1
2,28,41,0.742448,3.200006,3.124180,4.660488,0.049004,0.529022,0.528511,0.538335,1,1,1
3,25,49,1.000000,3.101582,3.159934,3.611285,0.044138,0.534171,0.534812,0.539764,1,1,1
4,22,69,1.000000,2.939584,3.789227,2.954523,0.067925,0.549753,0.563993,0.550004,1,1,1


### Visualize data

In [14]:
def logit_plot(dist, logit, RI):
    canvas = toyplot.Canvas(width=500, height=250)
    ax0 = canvas.cartesian(
        label="pooled data (function)",
        xlabel="Genetic dist.",
        ylabel="Logit function",
        grid=(1, 2, 0),
    )
    ax1 = canvas.cartesian(
        label="pooled data (observation)",
        xlabel="Genetic dist.",
        ylabel="RI",
        grid=(1, 2, 1),
    )

    # points are jittered on x-axis for visibility
    ax0.scatterplot(
        dist,
        logit,
        size=5,
        opacity=0.33,
        color=toyplot.color.Palette()[0],
    );
    ax1.scatterplot(
        dist,
        RI,
        size=10,
        opacity=0.2,
        marker="|",
        mstyle={
            "stroke": toyplot.color.Palette()[1],
            "stroke-width": 3,
        },
    );
    return canvas, (ax0, ax1)

In [139]:
logit_plot(SAMPLE.dist, SAMPLE.logit_b, SAMPLE.RI_pooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tbc24301e90fa44a7b69feeba0abaf16c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum

In [140]:
logit_plot(SAMPLE.dist, SAMPLE.logit, SAMPLE.RI_unpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tbbe1b07b329c45608fa03b9d2680d3a8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <circle r="2.5"

In [141]:
logit_plot(SAMPLE.dist, SAMPLE.logit_x, SAMPLE.RI_partpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t5f1b12a288fb45a09345735eadd07d96" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <cir

### Define models

In [142]:
def pooled_logistic(x, y, **kwargs):
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.HalfNormal('𝛽', sigma=10, shape=1)
        # 𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 𝛼 * (𝛽 ** x)
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
    
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [143]:
def unpooled_logistic(x, y, idx0, idx1, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0.values)
        sidx1 = pm.Data("spp_idx1", idx1.values)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        # 𝛽 = pm.Normal('𝛽', mu=1., sigma=10., shape=1)
        𝛽 = pm.HalfNormal('𝛽', sigma=10, shape=1)
        
        # link function
        effect = 𝛼 * ((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) ** x)
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(init = 'adapt_diag', **kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [144]:
def partpooled_logistic(x, y, idx0, idx1, gidx, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0)
        sidx1 = pm.Data("spp_idx1", idx1)
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        # 𝛽 = pm.Normal('𝛽', mu=1., sigma=10., shape=1)
        𝛽 = pm.HalfNormal('𝛽', sigma=10, shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        
        # linear model prediction
        effect = 𝛼 * ((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) ** x)
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood (normal distributed errors)
        y = pm.Bernoulli("y", p=logit, observed=y)

        # sample posterior, skip burnin
        trace = pm.sample(init = 'adapt_diag', **kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Functions to plot results

In [21]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [22]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

In [145]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=10000,
    draws=10000,
    target_accept=0.99,
    return_inferencedata=False,
    progressbar=True,
)

### Run three datasets under pooled model

In [146]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_pooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 363 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [147]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_unpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 378 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [148]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_partpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 383 seconds.
The number of effective samples is smaller than 25% for some parameters.


### Run three datasets under unpooled model

In [149]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_pooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 20244 seconds.
There were 3208 divergences after tuning. Increase `target_accept` or reparameterize.
There were 3729 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2705 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 3484 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_unpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_partpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


### Run three datasets under partpooled model

In [161]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_pooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3207 seconds.
There were 9876 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9808 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9979 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9738 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


In [162]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_unpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 5810 seconds.
There were 9135 divergences after tuning. Increase `target_accept` or reparameterize.
There were 8688 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9076 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9597 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9690075936476212, but should be close to 0.99. Try to increase the number of tuning steps.
The number of effective samples is smaller than 10% for some parameters.


In [163]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_partpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 4543 seconds.
There were 9388 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9317 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9647 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9618906375551507, but should be close to 0.99. Try to increase the number of tuning steps.
There were 9655 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In [164]:
partpooled_model_partpooled_data['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],-1.028,1.519,-3.830,1.313,0.074,0.052,425.0,425.0,451.0,642.0,1.01
𝜓_mean[1],-0.972,1.538,-3.922,1.395,0.074,0.052,437.0,437.0,471.0,601.0,1.01
𝜓_mean[2],-1.128,1.514,-3.890,1.192,0.073,0.052,429.0,429.0,464.0,637.0,1.01
𝜓_mean[3],-0.863,1.534,-3.804,1.548,0.074,0.052,433.0,433.0,459.0,670.0,1.01
𝜓_offset[0],0.336,0.959,-1.535,2.040,0.021,0.015,2103.0,2103.0,2103.0,3186.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...
logit[995],0.828,0.031,0.769,0.883,0.001,0.001,1906.0,1906.0,1944.0,2875.0,1.00
logit[996],0.935,0.031,0.883,0.990,0.001,0.001,1661.0,1619.0,1456.0,2215.0,1.00
logit[997],0.917,0.043,0.839,0.984,0.001,0.001,2655.0,2655.0,3144.0,3147.0,1.00
logit[998],0.890,0.036,0.823,0.956,0.001,0.001,2277.0,2238.0,2165.0,2682.0,1.00


In [165]:
toytrace(partpooled_model_partpooled_data['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t0753102e1a1447faa2b914f68dcf0589" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 56.86082803418562 149.92548435171386 L 60.13612893421049 149.92548435171386 L 63.41142983423542 149.92548435171386 L 66.68673073426035 149.92548435171386 L 69.96203163428524 149.92548435171386 L 73.23733253431016 150.0 L 76.51263343433507 150.0 L 79.78793433436 150.0 L 83.06323523438488 150.0 L 86.33853613440981 150.0 L 89.61383703443468 149.7764530551416 L 92.88913793445961 150.0 L 96.16443883448454 150.0 L 99.43973973450946 149.92548435171386 L 102.71504063453435 150.0 L 105.99034153455928 150.0 L 109.26564243458421 150.0 L 112.5409433346091 150.0 L 115.81624423463401 150.0 L 119.09154513465893 149.92548435171386 L 122.36684603468383 150.0 L 125.64214693470873 149.85096870342772 L 128.91744783473365 149.62742175856928 L 132.19274873475857 150.0 L 135.46804963478345 149.62742175856928 L 138.7433505348084 149.70193740685542 L 142.0186514348333 149.70193740685542 L 145.2939523348582 149.55290611028315 L 148.56925323488312 148.6587183308495 L 151.84455413490804 148.80774962742177 L 155.11985503493293 148.8822652757079 L 158.39515593495787 148.21162444113264 L 161.67045683498276 147.9880774962742 L 164.94575773500767 147.31743666169896 L 168.2210586350326 146.94485842026825 L 171.4963595350575 145.30551415797316 L 174.77166043508242 145.90163934426226 L 178.0469613351073 145.67809239940388 L 181.32226223513223 145.60357675111774 L 184.59756313515715 143.07004470938898 L 187.87286403518203 143.14456035767512 L 191.14816493520698 142.92101341281668 L 194.42346583523187 142.92101341281668 L 197.69876673525675 141.6542473919523 L 200.9740676352817 143.36810730253353 L 204.24936853530662 140.1639344262295 L 207.5246694353315 139.4187779433681 L 210.79997033535642 134.27719821162444 L 214.07527123538134 132.41430700447094 L 217.35057213540625 132.93591654247393 L 220.62587303543114 125.70789865871834 L 223.9011739354561 127.86885245901638 L 227.17647483548097 125.48435171385992 L 230.4517757355059 123.54694485842026 L 233.72707663553078 119.29955290611029 L 237.0023775355557 115.87183308494784 L 240.2776784355806 110.28315946348732 L 243.55297933560558 110.80476900149031 L 246.82828023563044 109.61251862891206 L 250.10358113565533 111.25186289120717 L 253.37888203568028 101.11773472429212 L 256.6541829357052 100.29806259314455 L 259.9294838357301 98.6587183308495 L 263.20478473575497 96.274217585693 L 266.4800856357799 89.71684053651266 L 269.7553865358048 88.4500745156483 L 273.03068743582975 86.43815201192251 L 276.30598833585464 81.44560357675111 L 279.5812892358796 85.61847988077497 L 282.85659013590447 80.92399403874813 L 286.13189103592936 80.62593144560357 L 289.4071919359543 87.40685543964233 L 292.6824928359792 77.19821162444114 L 295.95779373600413 79.43368107302533 L 299.2330946360291 76.3785394932936 L 302.5083955360539 70.34277198211625 L 305.78369643607886 70.11922503725783 L 309.0589973361038 58.7183308494784 L 312.33429823612863 56.63189269746647 L 315.60959913615363 56.11028315946349 L 318.8849000361785 54.32190760059612 L 322.1602009362034 50.81967213114754 L 325.4355018362283 60.804769001490314 L 328.71080273625324 63.26378539493294 L 331.98610363627813 55.81222056631893 L 335.2614045363031 57.37704918032786 L 338.53670543632796 75.03725782414307 L 341.81200633635285 85.54396423248883 L 345.08730723637774 97.61549925484353 L 348.36260813640274 112.66766020864382 L 351.63790903642763 129.28464977645305 L 354.9132099364525 136.21460506706407 L 358.18851083647746 140.38748

In [166]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_model_partpooled_data['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tddfb306a7e484a04b438fd9b98d23f79" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -1.6 -1.2 -0.8 -0.4 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [167]:
draw_velocity_dists(partpooled_model_partpooled_data['trace'], baseline = 1.2);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t899ece726d4a4658ae949d7870dc84db" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="111.32432616565323,250.0 112.4024855457418,250.0 113.48064492583035,250.0 114.5588043059189,250.0 115.63696368600748,250.0 116.71512306609603,250.0 117.7932824461846,250.0 118.87144182627316,250.0 119.9496012063617,250.0 121.02776058645026,250.0 122.1059199665388,250.0 123.18407934662739,250.0 124.26223872671594,250.0 125.3403981068045,250.0 126.41855748689306,250.0 127.49671686698161,250.0 128.57487624707016,250.0 129.65303562715872,250.0 130.73119500724727,250.0 131.80935438733584,250.0 132.8875137674244,250.0 133.96567314751297,250.0 135.04383252760152,250.0 136.12199190769007,250.0 137.20015128777865,250.0 138.2783106678672,250.0 139.35647004795575,250.0 140.43462942804433,250.0 141.51278880813288,250.0 142.59094818822143,250.0 143.66910756830998,250.0 144.74726694839856,250.0 145.8254263284871,250.0 146.90358570857566,250.0 147.9817450886642,250.0 149.0599044687528,250.0 150.13806384884134,250.0 151.2162232289299,250.0 152.29438260901847,250.0 153.37254198910702,250.0 154.45070136919557,250.0 155.52886074928412,250.0 156.6070201293727,250.0 157.68517950946125,250.0 158.7633388895498,250.0 159.84149826963838,250.0 160.91965764972693,250.0 161.99781702981548,250.0 163.07597640990406,250.0 164.15413578999258,250.0 165.23229517008116,250.0 166.3104545501697,250.0 167.3886139302583,250.0 168.46677331034684,250.0 169.5449326904354,250.0 170.62309207052394,250.0 171.70125145061252,250.0 172.77941083070107,250.0 173.85757021078962,250.0 174.9357295908782,250.0 176.01388897096675,250.0 177.09204835105533,250.0 178.17020773114388,250.0 179.24836711123243,250.0 180.32652649132098,250.0 181.40468587140953,250.0 182.48284525149808,250.0 183.56100463158663,250.0 184.63916401167518,250.0 185.71732339176376,250.0 186.79548277185233,250.0 187.87364215194089,250.0 188.95180153202944,250.0 190.029960912118,250.0 191.10812029220654,250.0 192.18627967229514,250.0 193.2644390523837,250.0 194.34259843247224,250.0 195.4207578125608,250.0 196.49891719264937,250.0 197.5770765727379,250.0 198.65523595282644,250.0 199.733395332915,250.0 200.81155471300357,250.0 201.88971409309212,250.0 202.96787347318067,250.0 204.04603285326925,250.0 205.12419223335783,250.0 206.20235161344635,250.0 207.28051099353493,250.0 208.3586703736235,250.0 209.43682975371206,250.0 210.51498913380064,250.0 211.5931485138892,250.0 212.67130789397774,250.0 213.7494672740663,250.0 214.82762665415487,250.0 215.9057860342434,250.0 216.98394541433197,250.0 218.06210479442052,250.0 218.06210479442052,248.83915225572588 216.98394541433197,248.64103518439373 215.9057860342434,248.41421402394283 214.82762665415487,248.1554758654416 213.7494672740663,247.86141005523132 212.67130789397774,247.52842321050176 211.5931485138892,247.1527598127428 210.51498913380064,246.73052884010173 209.43682975371206,246.2577368263881 208.3586703736235,245.73032763911957 207.28051099353493,245.14422915114517 206.20235161344635,244.49540684030273 205.12419223335783,243.77992419032438 204.04603285326925,242.99400958576675 202.96787347318067,242.13412919698152 201.88971409309212,241.19706514190395 200.81155471300357,240.17999799448597 199.733395332915,239.08059249059602 198.65523595282644,237.89708506757833 197.5770765727379,236.62837167049037 196.49891719264937,235.27409407383917 195.4207578125608,233.8347228101743 194.34259843247224,232.31163467387722 193.2644390523837,230.70718268731153 192.18627967229514,229.02475638391485 191.10812029220654,227.26883

### Assess model fit

In [165]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val   

In [166]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [167]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

0.9384409617057597

In [168]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

0.6105422350829149

In [177]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-24.454868204001738

In [179]:
# Raftery et al. (2007) holds that this value is better.  R+M disagree?
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-12.49733802346165

In [218]:
pooled_az = az.from_pymc3(trace = pooled_sub['trace'], model = pooled_sub['model'])
unpooled_az = az.from_pymc3(trace = unpooled_sub['trace'], model = unpooled_sub['model'])
partpooled_az = az.from_pymc3(trace = partpooled_sub['trace'], model = partpooled_sub['model'])

In [168]:
az_pooled_model_pooled_data = az.from_pymc3(trace = pooled_model_pooled_data['trace'], 
                                            model = pooled_model_pooled_data['model'])
az_pooled_model_unpooled_data = az.from_pymc3(trace = pooled_model_unpooled_data['trace'], 
                                            model = pooled_model_unpooled_data['model'])
az_pooled_model_partpooled_data = az.from_pymc3(trace = pooled_model_partpooled_data['trace'], 
                                            model = pooled_model_partpooled_data['model'])
az_unpooled_model_pooled_data = az.from_pymc3(trace = unpooled_model_pooled_data['trace'], 
                                            model = unpooled_model_pooled_data['model'])
az_unpooled_model_unpooled_data = az.from_pymc3(trace = unpooled_model_unpooled_data['trace'], 
                                            model = unpooled_model_unpooled_data['model'])
az_unpooled_model_partpooled_data = az.from_pymc3(trace = unpooled_model_partpooled_data['trace'], 
                                            model = unpooled_model_partpooled_data['model'])
az_partpooled_model_pooled_data = az.from_pymc3(trace = partpooled_model_pooled_data['trace'], 
                                            model = partpooled_model_pooled_data['model'])
az_partpooled_model_unpooled_data = az.from_pymc3(trace = partpooled_model_unpooled_data['trace'], 
                                            model = partpooled_model_unpooled_data['model'])
az_partpooled_model_partpooled_data = az.from_pymc3(trace = partpooled_model_partpooled_data['trace'], 
                                            model = partpooled_model_partpooled_data['model'])

In [169]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "pooled_model_unpooled_data": az_pooled_model_unpooled_data, 
            "pooled_model_partpooled_data": az_pooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_pooled_data,0,-291.722,2.01309,0,0.880317,20.9674,0,False,log
pooled_model_unpooled_data,1,-330.841,2.03463,39.1188,0.0749273,20.264,29.1499,False,log
pooled_model_partpooled_data,2,-336.819,2.04473,45.0971,0.0447562,20.6973,29.7582,False,log


In [170]:
az.compare({"unpooled_model_pooled_data": az_unpooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "unpooled_model_partpooled_data": az_unpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
unpooled_model_pooled_data,0,-292.354,10.9141,0,0.87745,20.451,0,True,log
unpooled_model_unpooled_data,1,-331.318,10.897,38.9643,0.0826055,20.8147,29.2304,False,log
unpooled_model_partpooled_data,2,-337.754,6.63364,45.4,0.0399447,20.4318,29.8222,False,log


In [171]:
az.compare({"partpooled_model_pooled_data": az_partpooled_model_pooled_data,
            "partpooled_model_unpooled_data": az_partpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
partpooled_model_pooled_data,0,-289.544,18.7623,0,0.913107,21.1126,0,False,log
partpooled_model_unpooled_data,1,-334.048,19.2282,44.5035,0.0623434,21.2682,29.248,False,log
partpooled_model_partpooled_data,2,-342.902,18.8178,53.3579,0.0245494,21.34,29.7038,False,log


In [172]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_pooled_data,0,-291.722,2.01309,0,0.901757,20.1314,0,False,log
unpooled_model_unpooled_data,1,-331.318,10.897,39.5957,0.0659336,21.1767,29.2068,False,log
partpooled_model_partpooled_data,2,-342.902,18.8178,51.1798,0.0323095,21.489,29.9336,False,log
